#### Progetto di corso per APPLICAZIONI DELL'INTELLIGENZA ARTIFICIALE (AA 2024-2025)
#### Stud: Marzio Della Bosca


Jupyter Notebook con la funzione di estrarre le feature, tramite catch22 e tsfel, dai dati contenuti nel dataset [Motion Sense](https://paperswithcode.com/dataset/motionsense)

In [1]:
import tsfel
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import time
import os
from aeon.transformations.collection.feature_based import Catch22

# (label-classe): 0-Downstair, 1-Upstair, 2-Walking, 3-Jogging, 4-Sitting, 5-Standing

In [2]:
motionP = pd.DataFrame(np.load('./MOTION_data/MotionSense.npy'))
motion = motionP.copy()

motion.columns = ['x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro', 'activity', 'subject', 'weight', 'height', 'age', 'gender', 'trial']
motion = motion.drop('weight', axis=1)
motion = motion.drop('height', axis=1)
motion = motion.drop('age', axis=1)
motion = motion.drop('gender', axis=1)
motion = motion.drop('subject', axis=1)
motion = motion.drop('trial', axis=1)

print(f"Shape motion dataframe: {motion.shape}")

Shape motion dataframe: (1412865, 7)


In [3]:
def windowing(data, window_size):
    data = pd.DataFrame(data)
    X, y = [], []

    for i in range(0, len(data) - window_size, window_size):
        window = data.iloc[i:i+window_size]
        X.append(window.values)
        # Calcola la moda solo sull'ultima colonna
        label_mode = window.iloc[:, -1].mode()[0]  # questo sistema mi serve quando dataset non è diviso in classi
        y.append(label_mode)                   

    return np.array(X), np.array(y)

def extract_catch22_features(data):
    # data shape: (n_samples, window, n_features)
    n_samples, window, n_channels = data.shape
    catch22 = Catch22()
    features = np.zeros((n_samples, 22, n_channels)) # prealloco la memoria in ram, velocizzo di molto i calcoli sfruttando 
                                                     # meglio anche le funzioni messe a disposizione da Numpy e Pandas

    for i in tqdm(range(n_samples), desc="Estrazione CATCH22"):
        for ch in range(n_channels):
            series = data[i, :, ch]
            if np.all(series == series[0]):
                series = series + np.random.normal(0, 1e-6, size=series.shape)
            try:
                feats = catch22.fit_transform([series]).flatten()
            except Exception:
                feats = np.full(22, np.nan)
            features[i, :, ch] = feats

    return features

def extract_tsfel_features(data, cfg_kind):
    # Carica la configurazione di default
    cfg = tsfel.get_features_by_domain(cfg_kind) # ovvero informazioni per domini 'statistical', 'temporal', 'spectral'
    # Calcola il numero di feature con un test rapido (primo esempio e un solo canale) (sempre per ottimizzare come per catch22)
    # data shape: (n_samples, window, n_features)
    n_samples, window, n_channels = data.shape
    test_feats = tsfel.time_series_features_extractor(cfg, pd.DataFrame(data[0, :, 0]), verbose=0)
    n_feat = test_feats.shape[1]
    features_3d = np.zeros((n_samples, n_feat, n_channels))

    for i in tqdm(range(n_samples), desc="Estrazione TSFEL"+ cfg_kind):
        for ch in range(n_channels):  # per ogni canale (6)
            signal = pd.DataFrame(data[i, :, ch])  # un canale come serie temporale
            feats = tsfel.time_series_features_extractor(cfg, signal, verbose=0, n_jobs=-1)
            features_3d[i, :, ch] = feats.values.flatten()
    
    return features_3d

In [4]:
window_size = 40 # imposto una finestra a 2 secondi, motion è campionato a 20 Hz
motion_raw_w40, motion_labels_w40 = windowing(motion, 40)

print(f"Shape Moby raw: {motion_raw_w40.shape}")
print(f"Shape Moby labels: {motion_labels_w40.shape}")

Shape Moby raw: (35321, 40, 7)
Shape Moby labels: (35321,)


In [5]:
np.save('motion_labels_40.npy', motion_labels_w40)
motion_raw_w40 = motion_raw_w40[:, :, :-1]
print(f"Shape Moby raw: {motion_raw_w40.shape}")

Shape Moby raw: (35321, 40, 6)


In [8]:
# Estraggo feature da catch22, non scalo ancora i dati in quanto non voglio fare estrazioni su segnali "ammortizzati" e quindi non reali
startT = time.time()
X_catch22 = extract_catch22_features(motion_raw_w40)
print(f"Tempo di estrazione feature catch22: {time.time() - startT:.2f} secondi")

Estrazione CATCH22: 100%|██████████| 35321/35321 [02:09<00:00, 271.82it/s]

Tempo di estrazione feature catch22: 129.95 secondi


In [ ]:
print(f"Shape X_catch22: {X_catch22.shape}")
np.save('motion_catch22_40.npy', X_catch22)

In [10]:
startT = time.time()
X_tsfel_stat = extract_tsfel_features(motion_raw_w40, 'statistical') # qui riuscirei a farlo completo, ma voglio avere dati coerenti con altri dataset 
                                                                     # con cui non riesco a calcolare tutti i domini, avendo shape diverse e problemi di 
                                                                     # aggregazione
print(f"\nTempo di estrazione feature tesfel con statistical: {time.time() - startT:.2f} secondi")
startT = time.time()
X_tsfel_temp = extract_tsfel_features(motion_raw_w40, 'temporal')
print(f"\nTempo di estrazione feature tesfel con temporal: {time.time() - startT:.2f} secondi")

Estrazione TSFELstatistical: 100%|██████████| 35321/35321 [06:39<00:00, 88.51it/s]



Tempo di estrazione feature tesfel con statistical: 399.05 secondi


Estrazione TSFELtemporal: 100%|██████████| 35321/35321 [03:15<00:00, 180.52it/s]


Tempo di estrazione feature tesfel con temporal: 195.66 secondi


In [11]:
X_tsfel_concat = np.concatenate((X_tsfel_stat, X_tsfel_temp), axis=1)
print("Shape concatenata:", X_tsfel_concat.shape)
np.save('motion_tsfel_40.npy', X_tsfel_concat)

Shape concatenata: (35321, 45, 6)
